In [1]:
!pip install xformers
!pip install transformers[torch]
!pip install datasets
!pip install  torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import set_seed,  GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, Trainer, TrainingArguments
from torch import tensor, numel
import pandas as pd
from datasets import Dataset

set_seed(42)

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
import time
import copy
import os
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import math
import random
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import gc
import torchtext
from tqdm import tqdm

# Misc.
import warnings
warnings.filterwarnings('ignore')

import json

In [4]:
MODEL_NAME = 'bert-base-cased'

sentiment_int = {'Quary': 0,
                 'Email': 1,
                 'Random': 2,
                 'Task': 3 ,
                 'Youtube': 4  }


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

RANDOM_SEED = 16

tokenizer_bert = BertTokenizer.from_pretrained(MODEL_NAME)

MAX_LEN = 50

batch_size = 16

loss_fn = nn.CrossEntropyLoss().to(device)

classes = ['Quary' , 'Email', 'Random' , 'Task' , 'Youtube'  ]

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Chatbot/Data/Data-large.csv")

In [6]:
class ChatbotDataset(Dataset):
    # Constructor Function
    def __init__(self, text, intend, tokenizer, max_len = 50):

        self.text = text
        self.intend = intend
        self.tokenizer = tokenizer
        self.max_len = max_len

    # Length magic method
    def __len__(self):
        return len(self.text)

    # get item magic method
    def __getitem__(self, item):

        text_ = str(self.text[item])
        intend_ = self.intend[item]

        # Encoded format to be returned
        encoding = self.tokenizer.encode_plus(
            text_,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text_,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'intend': torch.tensor(intend_, dtype=torch.long)
        }


def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = ChatbotDataset(
        text=df["text"].to_numpy(),
        intend=df["category"].to_numpy(),
        tokenizer=tokenizer,

    )

    return DataLoader(
        ds,
        batch_size=batch_size,

    )


In [7]:
df_train, df_val= train_test_split(df, test_size=0.15, random_state=RANDOM_SEED)

In [8]:
train_data_loader = create_data_loader(df_train, tokenizer_bert, MAX_LEN, batch_size)
val_data_loader = create_data_loader(df_val, tokenizer_bert, MAX_LEN, batch_size)


In [9]:
class SentimentClassifier(nn.Module):

    # Constructor class
    def __init__(self, n_classes = 5):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)


    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask,
          return_dict=False
        )
        #  Add a dropout layer
        output = self.drop(pooled_output)
        return self.out(output)


def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples,bert = True):



    model = model.train()
    losses = []
    correct_predictions = 0

    for d in tqdm(data_loader):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["intend"].to(device)


        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        # Backward prop
        loss.backward()

        # Gradient Descent
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)


def eval_model(model, data_loader, loss_fn, device, n_examples):

    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in tqdm(data_loader):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["intend"].to(device)

            # Get model ouptuts
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)


In [10]:
PATH =  "/content/drive/MyDrive/Chatbot/Models"

model_path = os.path.join(PATH,"Bert_model.pth")

In [11]:
model_trained = SentimentClassifier(len(sentiment_int))

In [12]:


if torch.cuda.is_available() == False:

    print("Cuda not available")
    model_trained.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

else:

    print("Cuda available")
    model_trained.load_state_dict(torch.load(model_path))


model_trained = model_trained.to(device)


Cuda available


In [13]:
val_acc, val_loss = eval_model(
model_trained,
val_data_loader,
loss_fn,
device,
len(df_val)
)

100%|██████████| 5/5 [00:02<00:00,  1.74it/s]


In [14]:
val_acc

tensor(0.9857, device='cuda:0', dtype=torch.float64)

In [15]:
tokenizer_bert_ = BertTokenizer.from_pretrained(MODEL_NAME)

In [16]:
text_ = "Google search for the symptoms of common illnesses."



tokenizer_bert_ = BertTokenizer.from_pretrained(MODEL_NAME)

def process_text_bert(s):

    return  tokenizer_bert_.encode_plus(
        s,
        add_special_tokens=True,
        max_length= 50,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )


def pred_bert(model_trained,encoding,flag):

    model_trained = model_trained.eval()

    with torch.no_grad():

        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        # targets = encoding["intend"].to(device)

        # Get model ouptuts
        outputs = model_trained(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs, dim=1)

        if(flag == True):

            preds_cpu = preds.cpu()

            return classes[preds_cpu]

        else:

            return classes[preds]









In [17]:
data_ = {"Quary" : "Google search for the symptoms of common illnesses.",
         "Youtube" : "I want to open YouTube and watch magic tricks.",
         "Random" : "What's the best way to learn about other cultures?" ,
         "Email" : "Email Jennifer regarding the team meeting" ,
         "Task" : "Remind me to call mom in 30 minutes." ,}


for ac,text_ in data_.items():

    encoding_ = process_text_bert(text_)
    flag = torch.cuda.is_available()

    p = pred_bert(model_trained,encoding_,flag)

    print(ac,p)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Quary Quary
Youtube Youtube
Random Random
Email Email
Task Task


In [18]:
loaded_model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Chatbot/Output_GPT_2_2')


tokenizer_gpt = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer_gpt.pad_token = tokenizer_gpt.eos_token

generator = pipeline('text-generation', model=loaded_model, tokenizer=tokenizer_gpt )

In [19]:
CONVERSION_PROMPT = "Q&A\n"
CONVERSION_TOKEN = "Answer: "
text_sample = 'who is spiderman?'
conversion_text_sample = f'{CONVERSION_PROMPT}Question: {text_sample}\n{CONVERSION_TOKEN}'

print(conversion_text_sample)

Q&A
Question: who is spiderman?
Answer: 


In [20]:
conversion_text_sample

'Q&A\nQuestion: who is spiderman?\nAnswer: '

In [21]:
texts = ['What makes you sad?',
         "go on comrade",
        "who is Batman?",
         "what happened to my happines?",
         "Do a chatbot like you likes chocolates?",
         "Apple or microsoft?"
         "do you think Ai will replace us?",
         "I am feeling sad",
         "How to propose my crush?"

         ]


data = []

results = []
answer = []

for t in texts:

    sample_= f'{CONVERSION_PROMPT}Question: {t}\n{CONVERSION_TOKEN}'

    data.append(sample_)

for i in data:

    res = generator(
        i, temperature=0.5,
        max_length=len(tokenizer_gpt.encode(i)) + 80
    )[0]['generated_text']



    ans_ = res.split("\n")[2][9:]

    print()
    print(ans_)
    print()



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sad is a subjective experience, and its keys can vary from person to person. It can be a source of sadness. Try taking deep breaths and focusing on positive outcomes. You've got this!



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



I'm feeling down, and I need a good laugh. Share your funniest joke!



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


I don't eat chocolate, but I can provide recipes and cooking tips. What about you? Tell me, what do you like to do in your free time?



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



I don't have preferences, but I can recommend a good alternative. What do you call a bear with no teeth? A gummy bear! How about a gummy bear with no teeth? A gummy bear! What do you call a rubber band with no teeth? A rubber band! What about a rubber band with no teeth? A rubber band! What about a rubber band without teeth?



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



I'm feeling sad right now. Can you cheer me up? 🤗 I'm here to chat and make you smile! Tell me, what's been on your mind? 🤗 I don't have personal interests, but I'm here to help you! 🤗 Tell me, what's been on your mind? 🤗 I don't have personal interests, but I'm


Her name is Suzuka Miyazaki, and she's a famous actress. How about a romantic comedy that'll have you laughing?



In [23]:
def process_text_gpt(t):

    return f'{CONVERSION_PROMPT}Question: {t}\n{CONVERSION_TOKEN}'

def pred_gpt(text):

    res = generator(
        text, temperature=0.5,
        max_length=len(tokenizer_gpt.encode(text)) + 30
    )[0]['generated_text']




    ans_ = res.split("\n")[2][9:] + " . . . "

    return ans_

In [24]:
s = "hello, how are you?"
s = process_text_gpt(s)

In [25]:
res = generator(
s, temperature=0.5,
max_length=len(tokenizer_gpt.encode(s)) + 30
)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [26]:
res

"Q&A\nQuestion: hello, how are you?\nAnswer:  I'm just a chatbot, so I don't have a physical presence in chatrooms. I'm designed to provide information and assist with tasks,"

In [27]:
res.split("\n")[2][9:]

"I'm just a chatbot, so I don't have a physical presence in chatrooms. I'm designed to provide information and assist with tasks,"

In [28]:
s = "hello, how are you?"
s = process_text_gpt(s)
s = pred_gpt(s)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [29]:
s

"I'm just a chatbot, so I don halftime show appearances. What attainable speed is the use of voice chat? . . . "

In [30]:
data = ["hello, how are you?",
        "give me the ring" ,
        "who is fast tiger or cat?",
        "why is sleep late is good",
        "Do chatbots have emotions?",
        "do Ai feels love?",
        "Do chatbot feels lonely"]


for d in data:


    s = process_text_gpt(d)
    s = pred_gpt(s)

    print()
    print("*"*100)
    print()
    print(d)
    print()
    print(s)
    print("*"*100)
    print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



****************************************************************************************************

hello, how are you?

I'm just a chatbot, so I don't have a real-life connection. I'm designed to provide information and assist with tasks, so . . . 
****************************************************************************************************



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



****************************************************************************************************

give me the ring

Why did the chicken cross the road? To get to the other side!!!! Did it make a sound? Did it make a sound? . . . 
****************************************************************************************************



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


****************************************************************************************************

why is sleep late is good

It's one of those things that doesn't always involve waking up early. It can be a source of stress. Try taking a deep breath and focusing . . . 
****************************************************************************************************



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



****************************************************************************************************

Do chatbots have emotions?

Empathy is a matter of personal preference, and its absence can lead to misunderstanding. It is crucial to understand and speak with others. What works best . . . 
****************************************************************************************************



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



****************************************************************************************************

do Ai feels love?

Ai doesn't have feelings of love, but she can be a source of love for someone. They may share the same feelings, or just different goals . . . 
****************************************************************************************************


****************************************************************************************************

Do chatbot feels lonely

I don't experience loneliness, but chatting with you can help. You've got this! What helps you?  I don't experience loneliness, but . . . 
****************************************************************************************************



In [31]:
final_df = df_val.copy()

In [32]:
final_df.sample(frac = 1)

,text,intend,category
189,"Hi, how's your day going?",Random,2
202,Who is your favorite actor?,Random,2
257,Tell me about a recent accomplishment.,Random,2
325,Set an alarm to practice the piano at 4 PM daily.,Task,3
324,Add 'Prepare for the interview' as a task for ...,Task,3
...,...,...,...
301,Create a task to book flight tickets for the v...,Task,3
393,Search YouTube for DIY crafting ideas.,Youtube,4
179,Compose an email to Mark about the team's achi...,Email,1
379,I want to open YouTube and watch stand-up comedy.,Youtube,4


In [33]:
classes

['Quary', 'Email', 'Random', 'Task', 'Youtube']

In [34]:
def Quary_F():

    print("Do google")

def Email_F():

    print("Open Email")

def Random_F(s):

    s = process_text_gpt(s)
    s = pred_gpt(s)
    print(s)

def Task_F():

    print("To Task")

def Youtube_F():

    print("Do Youtube")

In [35]:
input_ = []
intend_ = []


flag = torch.cuda.is_available()

def main_loop_test(n = 10):

    for t,ind in zip(final_df["text"][n:n+10] , final_df["intend"][n:n+10]):

        print()
        print("*"*100)
        print()
        print("Actual Class ", ind)
        print()
        print(t)
        print()

        encoding_ = process_text_bert(t)

        p = pred_bert(model_trained,encoding_,flag)

        if(p == "Quary"):

            Quary_F()

        elif(p == "Email"):

            Email_F()

        elif(p == "Youtube"):

            Youtube_F()

        elif(p == "Task"):

            Task_F()

        else:

            Random_F(t)

        print()
        print("*"*100)
        print()













In [153]:
main_loop_test(10)


****************************************************************************************************

Actual Class  Random

What's your favorite app?



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I don't have preferences, but I can recommend apps based on your interests. What would you do if you won the lottery? . . . 

****************************************************************************************************


****************************************************************************************************

Actual Class  Email

Send an email to Sarah regarding the project update.

Open Email

****************************************************************************************************


****************************************************************************************************

Actual Class  Quary

How to search for video tutorials on YouTube?

Do Youtube

****************************************************************************************************


****************************************************************************************************

Actual Class  Random

Do you believe in destiny?



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Belief in destiny is a matter of personal belief and cultural perspective. It may not always lead to happiness, but it can be a powerful motivator and . . . 

****************************************************************************************************


****************************************************************************************************

Actual Class  Youtube

I want to open YouTube and watch stand-up comedy.

Do Youtube

****************************************************************************************************


****************************************************************************************************

Actual Class  Quary

Find reliable sources of information on the internet.

Do google

****************************************************************************************************


****************************************************************************************************

Actual Class  Random

How do you like to spend your f

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I don't have free time, but I'm here to help you! What helps you?  I don't have free time, but I'm . . . 

****************************************************************************************************


****************************************************************************************************

Actual Class  Quary

Find information on classic literature on the web.

Do google

****************************************************************************************************


****************************************************************************************************

Actual Class  Youtube

Open YouTube and play a science documentary.

Do Youtube

****************************************************************************************************


****************************************************************************************************

Actual Class  Task

Remind me to send the monthly report at 3 PM.

To Task

*************************************

In [154]:
main_loop_test(52)


****************************************************************************************************

Actual Class  Task

Remind me to review the project proposal at 2 PM.

To Task

****************************************************************************************************


****************************************************************************************************

Actual Class  Youtube

Search for workout routines on YouTube.

Do Youtube

****************************************************************************************************


****************************************************************************************************

Actual Class  Task

Add 'Prepare for the interview' as a task for this week.

To Task

****************************************************************************************************


****************************************************************************************************

Actual Class  Youtube

Search for worko

In [37]:
# Using Gpu
main_loop_test(40)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



****************************************************************************************************

Actual Class  Random

Tell me a riddle.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sure! Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years . . . 

****************************************************************************************************


****************************************************************************************************

Actual Class  Random

Do you believe in aliens?



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The existence of aliens is a topic of scientific speculation, but there's no conclusive evidence either way. It's just a speculation based on what we know . . . 

****************************************************************************************************


****************************************************************************************************

Actual Class  Task

Schedule a reminder for the project kickoff meeting at 9 AM.

To Task

****************************************************************************************************


****************************************************************************************************

Actual Class  Youtube

Search YouTube for yoga and meditation sessions.

Do Youtube

****************************************************************************************************


****************************************************************************************************

Actual Class  Email

Write an email to Daniel 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I don't have memories, but I can describe a few memorable memories. . . . 

****************************************************************************************************


****************************************************************************************************

Actual Class  Email

I need to open my Gmail account.

Open Email

****************************************************************************************************


****************************************************************************************************

Actual Class  Task

Add 'Buy groceries for the party' to my to-do list for next week.

To Task

****************************************************************************************************


****************************************************************************************************

Actual Class  Random

Tell me about a recent accomplishment.

I don't have achievements, but I'm designed to assist with tasks and provide informa